In [16]:
import numpy as np
import pandas as pd

In [1]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "legislators/legislators-current.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

The DataFrame uses categorical dtypes for space efficiency: Reduces the memory load on your machine.
SQL =>
SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;

Call .groupby() & pass name of column you want to group on, which is "state". Then, use ["last_name"] to specify columns on which you want to perform the actual aggregation.

In [2]:
n_by_state = df.groupby("state")["last_name"].count()
n_by_state.head(10)

state
AK     3
AL     9
AR     6
AS     1
AZ    11
CA    54
CO    10
CT     7
DC     1
DE     3
Name: last_name, dtype: int64

Can pass a lot more than just a single column name to .groupby() as the first arg. Can also specify any of the following: 1. List of multiple column names; 2. Dict or pandas Series; 3. NumPy array or pandas Index, or an array-like iterable of these
Here’s an example of grouping jointly on two columns, which finds count of Congressional members broken out by state, then by gender: In SQL, it'd be =>
SELECT state, gender, count(name)
FROM df
GROUP BY state, gender
ORDER BY state, gender;

In [3]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F         2
       M         1
AL     F         2
       M         7
AR     F         0
                ..
WI     M         8
WV     F         2
       M         2
WY     F         2
       M         1
Name: last_name, Length: 112, dtype: int64

Prominent diff between pandas GroupBy & SQL query. SQL result contains three columns: state, gender,count
In pandas version, grouped-on columns are pushed into the MultiIndex of the resulting Series by default:

In [5]:
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
type(n_by_state_gender)

pandas.core.series.Series

In [6]:
n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

In [12]:
n_by_state_gender.array[:5]

<PandasArray>
[2, 1, 2, 7, 0]
Length: 5, dtype: int64

To more closely emulate SQL result and push the grouped-on columns back into columns in the result, you can use as_index=False:
Produces DF w/ three columns and a RangeIndex, vs. a Series w/ a MultiIndex.

In [14]:
print(df.groupby('party').get_group('Republican'))

     last_name first_name   birthday gender type state       party
12    Barrasso       John 1952-07-21      M  sen    WY  Republican
13      Wicker      Roger 1951-07-05      M  sen    MS  Republican
14     Collins      Susan 1952-12-07      F  sen    ME  Republican
15      Cornyn       John 1952-02-02      M  sen    TX  Republican
17      Graham    Lindsey 1955-07-09      M  sen    SC  Republican
..         ...        ...        ...    ...  ...   ...         ...
533       Hunt     Wesley 1981-11-13      M  rep    TX  Republican
534    Kiggans   Jennifer 1971-06-18      F  rep    VA  Republican
537  Van Orden    Derrick 1969-09-15      M  rep    WI  Republican
538    Hageman    Harriet 1962-10-18      F  rep    WY  Republican
539   Ricketts       Pete 1964-08-19      M  sen    NE  Republican

[274 rows x 7 columns]


In [25]:
import datetime
# Returns a Series
# birthday = df['birthday'][:1]
birthday = df['birthday'][1]
print(birthday)
type(birthday)
print(birthday.year)


1958-10-13 00:00:00
1958


In [29]:
df['birth_year'] = df['birthday'].map(lambda x: x.year)

In [30]:
aggs = df.groupby('party')['birth_year'].agg([np.mean, np.std, np.var])
print(aggs)

                    mean        std         var
party                                          
Democrat     1962.992424  13.285400  176.501844
Independent  1953.666667  19.399313  376.333333
Republican   1964.835766  11.196809  125.368533


In [13]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

,state,gender,last_name
0,AK,F,2
1,AK,M,1
2,AL,F,2
3,AL,M,7
4,AR,F,0
...,...,...,...
107,WI,M,8
108,WV,F,2
109,WV,M,2
110,WY,F,2
